# **GEFFEJEV GENERATOR**

**Avtor**: Nina Mislej
<br />**Vpisna številka**: 63200016

## **Kratek opis delovanja in teorije**
Na hitro razložimo osnovne pojme, ki jih bomo potrebovali za dešifriranje našega besedila. Tako kot pri Hillovi šifri si lahko predstavljamo, kot da so črke **angleške abecede** označene z indeksi od 0 do 25. Vsakemu od teh števil priredimo dvojiško zaporedje.

|0 A  |1 B  |2 C  |3 D  |4 E  |5 F  |6 G  |7 H  |8 I  |9 J  |10 K |11 L |12 M |
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|00000|00001|00010|00011|00100|00101|00110|00111|01000|01001|01010|01011|01100|

|13 N |14 O |15 P |16 Q |17 R |18 S |19 T |20 U |21 V |22 W |23 X |24 Y |25 Z |
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|01101|01110|01111|10000|10001|10010|10011|10100|10101|10110|10111|11000|11001|

Geffejev generator je tip **tokovne šifre** in je sestavljen iz treh pomičnih registrov s povratno zanko: **LFSR1**, **LFSR2** in **LFSR3**. Izhodne bite označimo z **x<sub>1</sub>**, **x<sub>2</sub>** in **x<sub>3</sub>**. Najprej črko zapišemo v binarni obliki, tko kot piše v tabeli zgoraj. Nato mu po modulu števila 2 (⨁) prištejemo ključ, ki ga vrne generator. Generator deluje po Boolean funkciji.

### **Parametri v našem primeru**
F(x1, x2, x3) = (x1 ∧ x2) ⨁ (x2 ∧ x3) ⨁ x3

Izpišimo še tabelo vrednosti:
| x1 | x2 | x3 | F  |
| -- | -- | -- | -- |
| 0  | 0  | 0  | 0  |
| 0  | 0  | 1  | 1  |
| 0  | 1  | 0  | 0  |
| 0  | 1  | 1  | 0  |
| 1  | 0  | 0  | 0  |
| 1  | 0  | 1  | 1  |
| 1  | 1  | 0  | 1  |
| 1  | 1  | 1  | 1  |

Karakteristični polinomi LFSR-jev:
- p1(x) = x<sup>5</sup> + x<sup>2</sup> + 1  &emsp; &emsp; &nbsp;<span>&#10230;</span>&emsp; &emsp;           x<sub>1, i</sub> =  x<sub>1, i-2</sub> ⨁ x<sub>1, i-5</sub>
- p2(x)= x<sup>7</sup> + x + 1               &emsp; &emsp; &nbsp; &nbsp; <span>&#10230;</span>&emsp; &emsp;   x<sub>2, i</sub> =  x<sub>2, i-1</sub> ⨁ x<sub>2, i-7</sub>
- p3(x)= x<sup>11</sup> + x<sup>2</sup> + 1  &emsp; &emsp; &nbsp;<span>&#10230;</span>&emsp; &emsp;           x<sub>3, i</sub> =  x<sub>3, i-2</sub> ⨁ x<sub>3, i-11</sub>

Red prvega polinoma je 5, polinom je nerazcepen zato je perioda 2<sup>5</sup> - 1 = 31
<br />Red drugega polinoma je 7, polinom je nerazcepen zato je perioda 2<sup>7</sup> - 1 = 127
<br />Red tretjega polinoma je 11, polinom je nerazcepen zato je perioda spet 2<sup>11</sup> - 1 = 2047

Poleg tega za nas specifični primer vemo tudi, da se besedilo, ki smo ga prestregli začne z besedo: **CRYPTOGRAPHY**

## **Program za dešifriranje**
Najprej postavimo ustrezno okolje in preberemo vsebino šifrirane datoteke. Imamo 2 slovarja, `bin` preslika črko v njegovo binarno reprezentacijo in `bch` binarno število nazaj v črko. 
Definiramo tudi dve funkciji ki to preslikavo opravljata za celo besedo. Nato naredimo še funkcijo, ki opravi xor operacijo med obema binarnima nizoma

In [1]:
import numpy as np
file = open('input.txt', 'r') 
encrypted_text = file.readline()
known_text = "cryptography"
file.close()
length = len(encrypted_text)
mod = 26 # dolzina abecede - mogoce so posplositve

alphabet = 'abcdefghijklmnopqrstuvwxyz'
num = {alphabet[i]: i for i in range(mod)}
ltr = {i: alphabet[i] for i in range(mod)}
bin = {alphabet[i]: format(i,'05b') for i in range(mod)}
bch = {format(i,'05b'): alphabet[i] for i in range(mod)}

def str_to_bin(word):
    bin_word = ""
    for char in word:
        bin_word += bin[char]
    return bin_word

def bin_to_str(bin_word):
    word = ""
    for chunk in [bin_word[i:i+5] for i in range(0, len(bin_word), 5)]:
        word += bch[chunk]
    return word

def num_to_bin_list(i,m):
    return [int(n) for n in list(format(i,f"0{m}b"))]

def list_to_str(sequence):
    sequence = list(map(str, sequence))
    return "".join(str(i) for i in sequence)

def xor(string1, string2):
    return [(ord(a) ^ ord(b)) for a, b in zip(string1, string2)]

Ker vemo prvo besedo, dolgo 12 črk, lahko tako dobimo izhodne vrednosti funkcije za **prvih 12 bitov**.
Vzamemo 5 $\times$ 12 bitov šifriranega besedila in 12 črk poznane besede v bitnem zapisu.
Na teh dveh nizih opravimo ⨁ operacijo in tako dobimo izhodne vrednosti, ki jih želimo.

In [2]:
encrypted_string = encrypted_text[:len(known_text)*5]
known_string = str_to_bin(known_text)
print("This is the encrypted word:", encrypted_string)
print("This is the word in binary:", known_string)

F = xor(encrypted_string, known_string)

print("Output of function F for the first 12 letters:")
for i in range(0, len(F), 5):
    print(F[i:i+5])


This is the encrypted word: 011001110111111110011000111110101010111001111011000111111001
This is the word in binary: 000101000111000011111001101110001101000100000011110011111000
Output of function F for the first 12 letters:
[0, 1, 1, 1, 0]
[0, 1, 1, 0, 0]
[0, 0, 1, 1, 1]
[1, 0, 1, 1, 0]
[0, 0, 0, 1, 0]
[1, 0, 0, 0, 0]
[1, 0, 0, 1, 1]
[1, 1, 1, 1, 1]
[0, 1, 1, 1, 1]
[0, 0, 0, 1, 1]
[0, 1, 0, 0, 0]
[0, 0, 0, 0, 1]


Dobili smo izhodne bite naših LFSR enačb. Če pogledamo zgoraj zapisano Boolean funkcijo vidimo da se vrednost **x<sub>1</sub>** in  **x<sub>3</sub>** ujemata z vrednostjo funkcije v **75%** primerov. Prav tako se **x<sub>2</sub>** ujema v **50%** primerov. Izkoristimo to analizo tako, da poskušamo razbiti vsako šifro posebaj. Najprej bomo razbili **LFSR1**, nato **LFSR3** in na koncu še **LFSR2**.

Vemo, da ima prva enačba **32 možnih ključev**, saj bo ta dolžine 5 in tretja enačba **2048 možnih ključev** ker bo ta dolžine 11. Napišimo algoritma za generiranje vseh treh sekvenc.

In [3]:
def LFSR(eq, key, length):
    output = key.copy()
    if eq == 1:
        for i in range(len(key), length):
            output.append(output[i-2] ^ output[i-5])
    if eq == 3:
        for i in range(len(key), length):
            output.append(output[i-2] ^ output[i-11])
    if eq == 2:
        for i in range(len(key), length):
            output.append(output[i-1] ^ output[i-7])
    return output

Definirajmo še našo boolean funkcijo:

In [4]:
def GEFFE(key1, key2, key3, length):
    LFSR1 = LFSR(1, key1, length)
    LFSR2 = LFSR(2, key2, length)
    LFSR3 = LFSR(3, key3, length)

    output = []
    for (x1, x2, x3) in zip(LFSR1, LFSR2, LFSR3):
        output.append((x1 & x2) ^ (x2 & x3) ^ x3)
    return output
        

Sedaj lahko razbijemo šifro. Definiramo funkcijo, ki bo primerjala izhodne vrednosti vsakega registra posebaj z Boolean funkcijo. Parameter `p` poda s kakšno verjetnostjo se morata zaporedji ujemati, da vrne funkcija `True` oziroma `False`.

In [5]:
def compare(Y,F,p):
    common = 0
    for (y,f) in zip(Y,F):
         if y == f: common += 1
    if common >= p*len(F): return True
    else: return False        

Sedaj združimo vse dosedanje rezultate. Prva funkcija nam pretvori število v seznam ničel in enic dolžine m.

In [6]:
def attack(m, eq, keys1=None, keys3=None):
    keys = []
    if eq == 3 or eq == 1:
        for i in range(pow(2,m)):
            key = num_to_bin_list(i,m)
            Y = LFSR(eq, key, len(F))
            if compare(Y,F,0.70):
                    keys.append(key)
        return keys
    
    else:
        for i in range(pow(2,m)):
            for key1 in keys1:
                 for key3 in keys3:
                      key2 = num_to_bin_list(i,m)
                      Y = GEFFE(key1, key2, key3, len(F))
                      if compare(Y,F,1):
                           keys.append([key1, key2, key3])
        return keys

keys1 = attack(5, 1)
keys3 = attack(11, 3)
keys = attack(7, 2, keys1, keys3)
print("POSSIBLE KEYS: ", keys)

POSSIBLE KEYS:  [[[0, 1, 1, 1, 0], [1, 1, 0, 1, 0, 0, 1], [1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0]]]


Zadnji korak, ko dobimo vse možne ključe je, da besedilo dešifriramo.

In [7]:
def encrypt(keys, encrypted_text):
    for key in keys:
        output = GEFFE(key[0], key[1], key[2], len(encrypted_text))
        output = list_to_str(output)
        text = xor(output, encrypted_text)
        text = list_to_str(text)
        text = bin_to_str(text)
        print(text)

encrypt(keys, encrypted_text)

cryptographypriortothemodernagewaseffectivelysynonymouswithencryptiontheconversionofinformationfromareadablestatetoapparentnonsensetheoriginatorofanencryptedmessagealicesharedthedecodingtechniqueneededtorecovertheoriginalinformationonlywithintendedrecipientsbobtherebyprecludingunwantedpersonsevefromdoingthesamethecryptographyliteratureoftenusesaliceaforthesenderbobbfortheintendedrecipientandeveeavesdropperfortheadversarysincethedevelopmentofrotorciphermachinesinworldwariandtheadventofcomputersinworldwariithemethodsusedtocarryoutcryptologyhavebecomeincreasinglycomplexanditsapplicationmorewidespread


## **Rešitev**

Cryptography prior to the modern age was effectively synonymous with encryption the conversion of information from areadable state to apparent nonsense. The originator of an encrypted message Alice shared the decoding technique needed to recover the original in formation only with intended recipients Bob thereby precluding unwanted persons Eve from doing the same. The cryptography literature often uses Alice A for the sender Bob B for the intended recipient and Eve eavesdropper for the adversary. Since the development of rotor cipher machines in World War I and the advent of computers in World War II the methods used to carry out cryptology have become increasingly complex and its application more widespread.